In [45]:
import cvxpy as cp
import numpy as np
import warnings
import sys
from IPython.core.interactiveshell import InteractiveShell
import torch
import torch.optim as optim
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import matplotlib.pyplot as plt

import time
from tqdm import tqdm

np.random.seed(42)
torch.manual_seed(42)

warnings.filterwarnings("ignore")
InteractiveShell.ast_node_interactivity = "all"
sys.path.insert(1,'E:\\User\\Stevens\\Spring 2024\\PTO - Fairness\\myGit\\myUtils')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [46]:
def genData(num_data, num_features, num_items, seed=42, Q=100, dim=1, deg=1, noise_width=0.5, epsilon=0.1):
    rnd = np.random.RandomState(seed)
    n = num_data
    p = num_features
    m = num_items
    
    # Split the population into group A (1/4) and group B (3/4)
    group_A_size = n // 4
    group_B_size = n - group_A_size
    
    # Generate x with a bias for group A
    x = np.zeros((n, m, p))
    x[:group_A_size] = rnd.normal(0.5, 1, (group_A_size, m, p))  # Slightly higher mean for group A
    x[group_A_size:] = rnd.normal(0, 1, (group_B_size, m, p))   # Standard distribution for group B
    
    B = rnd.binomial(1, 0.5, (m, p))

    c = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            values = (np.dot(B[j], x[i, j].reshape(p, 1)).T / np.sqrt(p) + 3) ** deg + 1
            values *= 5
            values /= 3.5 ** deg
            epislon = rnd.uniform(1 - noise_width, 1 + noise_width, 1)
            values *= epislon
            
            # Introduce bias for c: Group A has slightly higher values, Group B slightly lower
            if i < group_A_size:
                values *= 1.1  # Increase c for Group A
            else:
                values *= 0.9  # Decrease c for Group B
            
            values = np.ceil(values)
            c[i, j] = values

    c = c.astype(np.float64)
    r = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            # Generate r using normal distribution, then clip to [0, 1] range
            r[i, j] = np.clip(rnd.normal(0.5, 0.2), 0, 1)
            
            # Introduce bias for r: Group A has slightly lower values, Group B slightly higher
            if i < group_A_size:
                r[i, j] -= 0.1  # Decrease r for Group A
            else:
                r[i, j] += 0.1  # Increase r for Group B
            
            # Clip again to ensure values remain in [0, 1] range
            r[i, j] = np.clip(r[i, j], 0, 1)

    return x, r, c, Q


class optModel:
    """
    This is a class for optimization models.
    """

    def __init__(self, r, c, Q, alpha=0.5):
        self.r = r
        self.c = c
        self.Q = Q
        self.alpha = alpha
        self.num_data = num_data
        self.num_items = num_items

        
    def __repr__(self):
        return 'optModel ' + self.__class__.__name__
    
    def setObj2(self, a, r, b, c, Q, epsilon=0.01, alpha=0.5):
        if alpha == 1:
            self.objective = cp.sum(cp.log(a * r + b * self.d + epsilon))
        else:
            self.objective = cp.sum(cp.power(a * r + b * self.d + epsilon, 1 - alpha)) / (1 - alpha)
        
        self.constraints = [
            cp.sum(cp.multiply(c, self.d)) <= Q
        ]
        
        self.problem = cp.Problem(cp.Maximize(self.objective), self.constraints)
        self.a = a
        self.r = r
        self.b = b
        self.c = c
        self.Q = Q
        self.epsilon = epsilon


    def setObj(self, r, c):

        if self.alpha == 1:
            self.objective = cp.sum(cp.log(cp.multiply(r, self.d)))
        else:
            self.objective = cp.sum(cp.power(cp.multiply(r, self.d), 1 - self.alpha)) / (1 - self.alpha)
        
        self.constraints = [
            self.d >= 0,
            cp.sum(cp.multiply(c, self.d)) <= self.Q
        ]
        self.problem = cp.Problem(cp.Maximize(self.objective), self.constraints)


    def solve(self, closed=False):
        """
        A method to solve the optimization problem for one set of parameters.

        Args:
            r (np.ndarray): The r parameter for the optimization
            c (np.ndarray): The c parameter for the optimization
            closed (bool): solving the problem in closed form

        Returns:
            tuple: optimal solution and optimal value
        """
        if closed:
            return self.solveC()

        self.d = cp.Variable(self.num_items)
        self.setObj(self.r, self.c)
        self.problem.solve(abstol=1e-9, reltol=1e-9, feastol=1e-9)
        opt_sol = self.d.value
        opt_val = self.problem.value


        # Dual values for the constraints
        lambdas = self.constraints[0].dual_value
        mus = self.constraints[1].dual_value
    
        return opt_sol, opt_val, lambdas, mus


    def solveC(self):
        """
        A method to solve the optimization problem in closed form for one set of parameters.

        Args:
            r (np.ndarray): The r parameter for the optimization
            c (np.ndarray): The c parameter for the optimization

        Returns:
            tuple: optimal solution and optimal value
        """
        r = self.r
        c = self.c
        if self.alpha == 1:
            raise ValueError("Work in progress")
        c = c.cpu().numpy() if isinstance(c, torch.Tensor) else c
        r = r.cpu().numpy() if isinstance(r, torch.Tensor) else r
        S = np.sum(c ** (1 - 1 / self.alpha) * r ** (-1 + 1 / self.alpha))
        opt_sol_c = (c ** (-1 / self.alpha) * r ** (-1 + 1 / self.alpha) * self.Q) / S
        opt_val_c = np.sum((r * opt_sol_c) ** (1 - self.alpha)) / (1 - self.alpha)

        return opt_sol_c, opt_val_c
    
    def solveC2(self):
        """
        A method to solve the optimization problem in closed form using the given formula.

        Returns:
            tuple: optimal solution and optimal value
        """
        if self.alpha == 1:
            raise ValueError("Work in progress")

        a = self.a
        r = self.r
        b = self.b
        c = self.c
        epsilon = self.epsilon
        Q = self.Q
        alpha = self.alpha

        b_inverse_alpha = b ** (-1 / alpha)
        c_over_b = c / b
        ar_plus_epsilon = a * r + epsilon

        S1 = np.sum(c_over_b * ar_plus_epsilon)
        S2 = np.sum(c_over_b ** (1 - 1 / alpha))

        d_star = b_inverse_alpha * (Q + S1) / S2 - ar_plus_epsilon * b_inverse_alpha / b

        opt_val_c2 = np.sum((a * r + b * d_star + epsilon) ** (1 - alpha)) / (1 - alpha)

        return d_star, opt_val_c2


    
class optDataset(Dataset):
    """
    This class is Torch Dataset class for optimization problems.
    """

    def __init__(self, features, costs, r, Q, alpha=0.5, closed=False):
        """
        A method to create a optDataset from optModel

        Args:
            model (optModel): optimization model
            features (np.ndarray): features
            c (np.ndarray): c of objective function
            r (np.ndarray): r of objective function
            Q (float): budget
            alpha (float): alpha of objective function
            closed (bool): solving the problem in closed form

        """
        self.feats = features
        self.costs = costs
        self.r = r
        self.Q = Q
        self.alpha = alpha
        self.closed = closed
        # Now store the dual values
        self.sols, self.objs, self.lambdas, self.mus = self._getSols()

    def _getSols(self):
        """
        A method to get the solutions and dual values of the optimization problem
        """
        opt_sols = []
        opt_objs = []
        dual_lambdas = []
        dual_mus = []
        
        for i in tqdm(range(len(self.costs))):
            sol, obj, lambdas, mus = self._solve(self.r[i], self.costs[i])
            opt_sols.append(sol)
            opt_objs.append([obj])
            dual_lambdas.append(lambdas)
            dual_mus.append(mus)
            
        return np.array(opt_sols), np.array(opt_objs), np.array(dual_lambdas), np.array(dual_mus)


    def  _solve(self, r, c):
        """
        A method to solve the optimization problem to get oan optimal solution with given r and c

        Args:
            r (np.ndarray): r of objective function
            c (np.ndarray): c of objective function

        Returns:
            tuple: optimal solution (np.ndarray), objective value (float)
        """
        self.model = optModel(r, c, self.Q, self.alpha)
        if self.closed:
            return self.model.solveC()
        else:
            return self.model.solve()

    def __len__(self):
        """
        A method to get data size

        Returns:
            int: the number of optimization problems
        """
        return len(self.costs)
    
    def __getitem__(self, index):
        return (
            torch.FloatTensor(self.feats[index]),  # x 
            torch.FloatTensor(self.costs[index]),  # c
            torch.FloatTensor(self.r[index]),      # r 
            torch.FloatTensor(self.sols[index]),   # optimal solution
            torch.FloatTensor(self.objs[index]),   # objective value
            torch.FloatTensor(self.lambdas[index]),  # dual value (lambdas)
            torch.FloatTensor([self.mus[index]])     # dual value (mus)
        )

    
class LinearRegressionModel(nn.Module):
    def __init__(self, num_items, num_features):
        super(LinearRegressionModel, self).__init__()
        self.num_items = num_items
        self.num_features = num_features
        self.linears = nn.ModuleList([nn.Linear(num_features, 1) for _ in range(num_items)])

    def forward(self, x):
        outputs = []
        for i in range(self.num_items):
            outputs.append(torch.sigmoid(self.linears[i](x[:, i, :])))
        return torch.cat(outputs, dim=1)
    

def regret(predmodel, optmodel, dataloader):
    """
    A function to evaluate model performance with normalized true regret

    Args:
        predmodel (nn): a regression neural network for cost prediction
        optmodel (optModel): an PyEPO optimization model
        dataloader (DataLoader): Torch dataloader from optDataSet

    Returns:
        float: true regret loss
    """
    # evaluate
    predmodel.eval()
    loss = 0
    optsum = 0
    # load data
    for data in dataloader:
        x, c, r, d, z  = data
        # cuda
        if next(predmodel.parameters()).is_cuda:
            x, c, r, d, z = x.cuda(), c.cuda(), r.cuda(), d.cuda(), z.cuda()
        # predict
        with torch.no_grad(): # no grad
            rp = predmodel(x).to("cpu").detach().numpy()
        # solve
        for j in range(rp.shape[0]):
            # accumulate loss
            loss += calRegret(optModel, c[j].to("cpu").detach().numpy(), rp[j], r[j].to("cpu").detach().numpy(),
                              z[j].item())
        optsum += abs(z).sum().item()
    # turn back train mode
    predmodel.train()
    # normalized
    return loss / (optsum + 1e-7)

def objValue(d, r, alpha=0.5):
    """
    A function to calculate objective value
    """
    if alpha == 1:
        return np.sum(np.log(np.multiply(r, d)))
    else:
        return np.sum(np.power(np.multiply(r, d), 1 - alpha)) / (1 - alpha)



def calRegret(optmodel, cost, pred_r, true_r, true_obj):
    """
    A function to calculate normalized true regret for a batch

    Args:
        optmodel (optModel): optimization model
        pred_cost (torch.tensor): predicted costs
        true_cost (torch.tensor): true costs
        true_obj (torch.tensor): true optimal objective values

    Returns:predmodel
        float: true regret losses
    """
    # opt sol for pred cost
    model = optmodel(pred_r, cost, Q, alpha=0.5)
    sol, _ = model.solve()
    # obj with true cost
    obj = objValue(sol, true_r, alpha=0.5)
    # loss
    loss = true_obj - obj
    return loss

# Define the visualization function
def visLearningCurve(loss_log, loss_log_regret):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4))

    ax1.plot(loss_log, color="c", lw=2)
    ax1.tick_params(axis="both", which="major", labelsize=12)
    ax1.set_xlabel("Iters", fontsize=16)
    ax1.set_ylabel("Loss", fontsize=16)
    ax1.set_title("Learning Curve on Training Set", fontsize=16)

    ax2.plot(loss_log_regret, color="royalblue", ls="--", alpha=0.7, lw=2)
    ax2.set_xticks(range(0, len(loss_log_regret), 2))
    ax2.tick_params(axis="both", which="major", labelsize=12)
    ax2.set_ylim(0, 0.10)
    ax2.set_xlabel("Epochs", fontsize=16)
    ax2.set_ylabel("Regret", fontsize=16)
    ax2.set_title("Learning Curve on Test Set", fontsize=16)

    plt.show()

In [47]:
num_data = 10
num_features = 10
num_items = 5

x, r, c, Q = genData(num_data, num_features, num_items)
optmodel = optModel(r, c, Q, alpha=0.5)

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, c_train, c_test, r_train, r_test = train_test_split(x, c, r, test_size=0.2, random_state=42)

# Create datasets and dataloaders
dataset_train = optDataset(x_train, c_train, r_train, Q, alpha=0.5, closed=False)
dataset_test = optDataset(x_test, c_test, r_test, Q, alpha=0.5, closed=False)
batch_size = 32
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

100%|██████████| 2/2 [00:00<00:00, 307.49it/s]


In [55]:
dataset_train[0]

(tensor([[-1.2609,  0.9179,  2.1222,  1.0325, -1.5194, -0.4842,  1.2669, -0.7077,
           0.4438,  0.7746],
         [-0.9269, -0.0595, -3.2413, -1.0244, -0.2526, -1.2478,  1.6324, -1.4301,
          -0.4400,  0.1307],
         [ 1.4413, -1.4359,  1.1632,  0.0102, -0.9815,  0.4621,  0.1991, -0.6002,
           0.0698, -0.3853],
         [ 0.1135,  0.6621,  1.5860, -1.2378,  2.1330, -1.9521, -0.1518,  0.5883,
           0.2810, -0.6227],
         [-0.2081, -0.4930, -0.5894,  0.8496,  0.3570, -0.6929,  0.8996,  0.3073,
           0.8129,  0.6296]]),
 tensor([5., 4., 8., 6., 7.]),
 tensor([0.8992, 0.7309, 0.5889, 0.6560, 0.3749]),
 tensor([6.0042, 7.6252, 1.5359, 3.0418, 1.2772]),
 tensor([15.4798]),
 tensor([1.3316e-10, 8.6852e-11, 7.0228e-10, 2.6338e-10, 8.6131e-10]),
 tensor([0.0774]))

In [51]:
import numpy as np
from scipy.linalg import solve

# Helper function to check matrix invertibility
def is_invertible(matrix):
    return np.linalg.cond(matrix) < 1 / np.finfo(matrix.dtype).eps

# Function to form the matrix system and solve it
def solve_rd_matrix_system(r, c, d_i, mu, lambda_i, Q, alpha = 0.5):
    n = len(r)
    
    # Step 1: Construct Hessian of the objective function (H_dd)
    A_elements = [alpha * r[i]**2 * (r[i] * d_i[i])**(-alpha - 1) for i in range(n)]
    H_dd = np.diag(A_elements)
    
    # Step 2: Construct Jacobians of inequality constraints (g(d;r)) and equality constraints (h(d;r))
    # Jacobian of g(d;r) = d >= 0 is -I
    B = -np.eye(n)
    
    # Jacobian of h(d;r) = sum(c * d) - Q is the vector of c_i's
    C = np.zeros((n, 1))
    for i in range(n):
        C[i, 0] = c[i]
    
    # Step 3: Construct D and M matrices
    D = np.diag(d_i)  # Diagonal matrix with d_i's
    M = np.zeros((1, n))
    M[0, :] = mu * c  # Row vector of mu * c
    
    # Step 4: Complementary slackness matrix with lambda_i
    Lambda = np.diag(lambda_i)
    
    # Step 5: Construct the full block matrix system
    LHS = np.block([
        [H_dd, B, C],  # First row block
        [Lambda, D, np.zeros((n, 1))],  # Second row block
        [M, np.zeros((1, n)), np.array([[np.sum(c * d_i) - Q]])]  # Third row block
    ])
    
    # Step 6: Construct the RHS vector
    # RHS vector contains v = -α r (r d)^{-α-1}, 0 for other conditions
    v = np.array([-alpha * r[i] * (r[i] * d_i[i])**(-alpha - 1) for i in range(n)])
    RHS = np.hstack([v, np.zeros(n), np.zeros(1)])
    
    # Step 7: Solve the matrix system
    if is_invertible(LHS):
        solution = solve(LHS, RHS)
        d_r_derivatives = solution[:n]
        lambda_r_derivatives = solution[n:2*n]
        mu_r_derivative = solution[2*n:]
        
        return d_r_derivatives


# Provided data
c = np.array([5., 3., 4., 8., 9.])  # c values
r = np.array([0.9055, 1.0000, 1.0000, 1.0000, 0.9991])  # r values
d_i = np.array([3.6203, 11.1061, 6.2473, 1.5618, 1.2329])  # Optimal d values
z = np.array([20.0043])  # Objective value (used for reference, not in matrix calculation)
dual_lambda = np.array([6.16273581e-10, 1.82440296e-10, 4.25497324e-10, 1.70830862e-09, 2.18849516e-09])  # Dual lambda values
dual_mu = 0.10002232440085013  # Dual mu value
Q = 100  # Equality constraint
alpha = 0.5  # Alpha value for fairness objective

# Solve the system with the given data
solution = solve_rd_matrix_system(r, c, Q, alpha, d_i, dual_mu, dual_lambda)



In [52]:
import numpy as np
def partial_derivative_d_i_wrt_r_i(c, r, Q, alpha):
    A = c**(-1/alpha) * r**(-1 + 1/alpha)
    S = np.sum(c**(1 - 1/alpha) * r**(-1 + 1/alpha))
    
    d_dr = []
    
    for i in range(len(r)):
        A_i = c[i]**(-1/alpha) * r[i]**(-1 + 1/alpha)
        partial_A_i_wrt_r_i = c[i]**(-1/alpha) * (-1 + 1/alpha) * r[i]**(-2 + 1/alpha)
        partial_S_wrt_r_i = c[i]**(1 - 1/alpha) * (-1 + 1/alpha) * r[i]**(-2 + 1/alpha)
        
        numerator = Q * (partial_A_i_wrt_r_i * S - A_i * partial_S_wrt_r_i)
        denominator = S**2
        
        derivative = numerator / denominator
        
        d_dr.append(derivative)

    return np.array(d_dr)

derivatives = partial_derivative_d_i_wrt_r_i(c, r, Q, alpha)

print("Partial derivatives of d_i with respect to r_i:", derivatives)

print(solution)

Partial derivatives of d_i with respect to r_i: [3.27446649 7.40576132 4.68611142 1.36666686 1.09709044]
[-0.03529868  2.27962759  0.84484312 -0.53880029 -0.63682593]


In [53]:
c = np.array([8., 6., 10., 9., 6.]) 
r = np.array([1.0000, 1.0000, 0.7353, 0.9085, 0.7466])  
d_i = np.array([2.6457, 4.7035, 1.2451, 1.8992, 3.5117]) 
z = np.array([15.3699])  # Objective value for reference
dual_lambda = np.array([3.51481403e-10, 2.02084120e-10, 7.82048660e-10, 5.05606257e-10, 2.61115052e-10])  # Dual lambda values
dual_mu = 0.07684988998398859  # Dual mu value
Q = 100
alpha = 0.5  

# Calculate the partial derivatives
derivatives = partial_derivative_d_i_wrt_r_i(c, r, Q, alpha)

# Print the results
print("Partial derivatives of d_i with respect to r_i:", derivatives)

Partial derivatives of d_i with respect to r_i: [2.08573851 3.37613329 1.48244495 1.73314022 3.71249268]


In [54]:
# Solve the system with the given data
solution = solve_rd_matrix_system(r, c, Q, alpha, d_i, dual_mu, dual_lambda)

# Output the results
if "error" in solution:
    print(solution["error"])
else:
    print("Derivatives of d with respect to r:", solution["d_r_derivatives"])
    print("Derivatives of lambda with respect to r:", solution["lambda_r_derivatives"])
    print("Derivative of mu with respect to r:", solution["mu_r_derivative"])

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices